In [12]:
from kgforge.core import KnowledgeGraphForge

In [13]:
import getpass
TOKEN = getpass.getpass()

In [14]:
BUCKET = "dke/kgforge"

forge = KnowledgeGraphForge(
    "../use-cases/prod-forge-nexus.yml",
    bucket=BUCKET,
    token=TOKEN
)

In [15]:
id_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/20fbc97a-fb26-43ff-8093-9136aab25dff"
self_ = "https://bbp.epfl.ch/nexus/v1/resources/dke/kgforge/_/https:%2F%2Fbbp.epfl.ch%2Fnexus%2Fv1%2Fresources%2Fdke%2Fkgforge%2F_%2F20fbc97a-fb26-43ff-8093-9136aab25dff"

id_other_bucket = "http://purl.obolibrary.org/obo/GO_0038048"
self_other_bucket = "https://bbp.epfl.ch/nexus/v1/resources/neurosciencegraph/datamodels/_/http:%2F%2Fpurl.obolibrary.org%2Fobo%2FGO_0038048"

all_ret = [
    (id_, "id", "same bucket"), 
    (self_, "self", "same bucket"), 
    (id_other_bucket, "id", "other bucket"),
    (self_other_bucket, "self", "other bucket"),
]

i = 0

for rs in [True, False]:
    for cb in [True, False]:
        for (el, type_, loc) in all_ret:

            print(f"Execution {i}", type_, loc, "Cross bucket", cb, "Retrieve source", rs)

            e = forge.retrieve(el, cross_bucket=cb, retrieve_source=rs)

            if not e:
                print("Not found")
            else:
                print("rev", e._store_metadata._rev)
            print("______________________")
            i += 1


Execution 0 id same bucket Cross bucket True Retrieve source True
rev 3
______________________
Execution 1 self same bucket Cross bucket True Retrieve source True
rev 3
______________________
Execution 2 id other bucket Cross bucket True Retrieve source True
rev 3
______________________
Execution 3 self other bucket Cross bucket True Retrieve source True
rev 3
______________________
Execution 4 id same bucket Cross bucket False Retrieve source True
rev 3
______________________
Execution 5 self same bucket Cross bucket False Retrieve source True
rev 3
______________________
Execution 6 id other bucket Cross bucket False Retrieve source True
<action> catch_http_error
<error> RetrievalError: resource 'http://purl.obolibrary.org/obo/GO_0038048' not found in project 'dke/kgforge'

Not found
______________________
Execution 7 self other bucket Cross bucket False Retrieve source True
<action> retrieve
<error> RetrievalError: Provided resource identifier https://bbp.epfl.ch/nexus/v1/resources/